<a href="https://colab.research.google.com/github/steph1793/SoccerCommentsSumm/blob/master/Data/New_Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
wd = webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.website.com")
print(wd.page_source)  # results

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [908 kB]
Get:14 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic/main Sour

In [2]:
!pip install XlsxWriter

     |████████████████████████████████| 143kB 2.8MB/s 


In [0]:
from selenium import webdriver
import numpy as np
import pandas as pd
import requests
import urllib.request
import time
import bs4
import lxml.etree as xml
import json
from bs4 import BeautifulSoup
import csv
import re
from selenium.common.exceptions import WebDriverException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from google.colab import files
from selenium.webdriver.common.keys import Keys
import xlsxwriter
from tqdm import tqdm

In [0]:
def ren_equipe(url):
  equipe = []
  i_split = url.split('/')
  i_equipe = i_split[7].split('-')
  if len(i_equipe) > 3:
    if i_equipe[0] == 'paris' or i_equipe[0] == 'saint' or i_equipe[0] == 'ac':
      del i_equipe[1]
    if i_equipe[1] == 'paris' or i_equipe[1] == 'saint' or i_equipe[1] == 'ac':
      del i_equipe[2]
  for j in range(2):
    if i_equipe[j] == 'ol': i_equipe[j] = 'lyon'
    elif i_equipe[j] == 'psg': i_equipe[j] = 'paris'
    elif i_equipe[j] == 'saint'or i_equipe[j] == 'asse': i_equipe[j] = 'saint etienne'
    elif i_equipe[j] == 'ac'or i_equipe[j] == 'gfc': i_equipe[j] = 'ajaccio'
    elif i_equipe[j] == 'om': i_equipe[j] = 'marseille'
  equipe.append(i_equipe[0][0].upper()+i_equipe[0][1:])
  equipe.append(i_equipe[1][0].upper()+i_equipe[1][1:])
  return equipe

In [0]:
def create_dataset_Commentaire_Sans_Resume(url,num_match,year):

  ### WEBDRIVER OPTIONS ###
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--ignore-certificate-errors')
  options.add_argument("--test-type")
  options.add_argument('--disable-browser-side-navigation')
  browser = webdriver.Chrome('chromedriver',options=options)
  

  for i, idx_match in zip(url, tqdm(range(len(url)))) :

    try:

      row_important = 0
      row = 0
      col = 0
      commentaire_avant_match = False
      row_commentaire_avant_match = 0
      index_match_equipe_saison = '-'.join(i.split('/')[6:])

      ### EXCEL FILES CREATION ###
      workbook = xlsxwriter.Workbook("CommentaireMatch"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_stats = xlsxwriter.Workbook("Stats"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_scorer = xlsxwriter.Workbook("Scorer"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_important = xlsxwriter.Workbook("important"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_avant_match = xlsxwriter.Workbook("avant_match"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_resume = xlsxwriter.Workbook("resume"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_phrase_but = xlsxwriter.Workbook("phrase_but"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")

      worksheet = workbook.add_worksheet()
      worksheet_stats = workbook_stats.add_worksheet()
      worksheet_scorer = workbook_scorer.add_worksheet()
      worksheet_important = workbook_important.add_worksheet()
      worksheet_avant_match = workbook_avant_match.add_worksheet()
      worksheet_resume = workbook_resume.add_worksheet()
      worksheet_phrase_but = workbook_phrase_but.add_worksheet()

      browser.get(i)
      time.sleep(1)
      compteur = 1

      ### REGULAR EXPRESSIONS ###
      a = 0
      apo='\''
      empty=''
      plus = "+"
      ex = "*"
      note=r'[0-9]+\/.[0-9]+'
      twitter=r'.*(PM).*(Twitter)?.*'
      compt = 0
      regex_time=r'[0-9]+\'\s?\+?[0-9]*'
      regex_if=r'\w'
      data={}
      data["moments"]=[]
      reg=r'.*But+.*'
      mid=r'45+?.*'
      regex_score_equipe1=r'([A-Z]+.*[0-9]’)'
      regex_score_equipe2=r'([0-9]+’.*[A-Z]+.*)'
      ### WEB PAGE NAVIGATION ###
      elem = browser.find_element_by_tag_name("body")
      no_of_pagedowns = 25
      while no_of_pagedowns:
          elem.send_keys(Keys.PAGE_DOWN)
          time.sleep(0.3)
          no_of_pagedowns-=1

      ### DATA LIST CREATION ###
      info = []
      info_stat = []
      info_score = []

      

      telem = browser.find_elements_by_class_name("TeamScore__identity")
      #equipe_dom= ""
      #equipe_ext= ""
      if len(telem)>0:
        equipe_dom = telem[0].text
        equipe_ext = telem[1].text
        if equipe_dom == "PSG":
          equipe_dom= "Paris"
        if equipe_ext == "PSG":
          equipe_ext= "Paris"
      else :
        equipe_dom, equipe_ext = ren_equipe(i)
      #SCORERS
      row_score1 = 0
      row_score2 = 0
      test= browser.find_elements_by_class_name("Scoreboard__goals")
      for texts in test:
        if(re.match(regex_score_equipe1,texts.text)):
          worksheet_scorer.write(row_score1, 0, texts.text)
          spli = texts.text.split(" ")
          if len(spli)==3:
            sentence = spli[0] + " " + spli[1] + " a marqué un but pour " + equipe_dom + " à la " + spli[2] + "."
          if len(spli)==2:
            sentence = spli[0] +  " a marqué un but pour " + equipe_dom + " à la " + spli[1] + "."
          worksheet_phrase_but.write(row_score1, 0, sentence)
          row_score1+=1
        if(re.match(regex_score_equipe2,texts.text)):
          worksheet_scorer.write(row_score2, 1, texts.text)
          spli = texts.text.split(" ")
          if len(spli)==3:
            sentence = spli[1] + " " + spli[2] + " a marqué un but pour " + equipe_ext + " à la " + spli[0] + "."
          if len(spli)==2:
            sentence = spli[1] +  " a marqué un but pour " + equipe_ext + " à la " + spli[0] + "."
          worksheet_phrase_but.write(row_score2, 1, sentence)
          row_score2+=1
      workbook_scorer.close()
      workbook_phrase_but.close()

      #GAME SCORE
      post_elems = browser.find_elements_by_class_name("TeamScore__team--winner") # TeamScore__score # TeamScore__team--looser # TeamScore__team--winner
      for texts in post_elems:
        info.append(texts.text)
      post_elems = browser.find_elements_by_class_name("TeamScore__team--looser") # TeamScore__score # TeamScore__team--looser # TeamScore__team--winner
      for texts in post_elems:
        info.append(texts.text) 
      if info != []:
        if info[0].isdigit():
          phrase = info[1] +" gagne le match contre " + info[2] + " sur un score de " + info[0]+"-"+info[3] + "."
        else :
          phrase = info[0] +" gagne le match contre " + info[3] + " sur un score de " + info[1]+"-"+info[2] + "."
      else :
        post_elems = browser.find_elements_by_class_name("TeamScore__name") 
        for texts in post_elems:
          info.append(texts.text)
        post_elems = browser.find_elements_by_class_name("TeamScore__score--ended") 
        for texts in post_elems:
          info.append(texts.text)
        phrase = info[0] +" fait match nul contre " + info[1] + " sur un score de " + info[2] + "."
      worksheet.write(row, col, phrase)
      row +=1

      #SENTENCES
      post_elems = browser.find_elements_by_class_name("grid__item")
      for texts in post_elems:
        if compteur >=0:
          if texts.text == "" or re.search(r'.*Notez.*',texts.text) or re.search(r'Top du match.*',texts.text) or re.search(r'Flop du match.*',texts.text) or re.search(r'.*note.*',texts.text) or re.search(r'.*[?].*',texts.text) or re.search(r'.*Twitter.*',texts.text): 
            a = a +1
          el= texts.find_elements_by_tag_name("img") 
          for em in el:
            if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/but.svg"  or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/carton_rouge.svg" or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/but_contre_son_camp.svg":
              worksheet_important.write(row_important, col, texts.text)
              print("IMPORTANT: ",texts.text)
              row_important +=1
            if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/sifflet_fin_2mt.svg":
              worksheet_resume.write(0, 0, texts.text.split("\n")[2])
            if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/sifflet_debut_1mt.svg":
              #print("DEBUT: ",texts.text)
              commentaire_avant_match = True
          if commentaire_avant_match == True :
            worksheet_avant_match.write(row_commentaire_avant_match, col, texts.text)
            #print(texts.text)
            row_commentaire_avant_match += 1
          else :
            worksheet.write(row, col, texts.text)
            #print(texts.text)
            row +=1
        compteur +=1
                             
      workbook.close()
      workbook_important.close()
      workbook_resume.close()
      workbook_avant_match.close()
      #STATISTICS
      row_stat = 0
      column_stat = 0

      browser.get(i+"#stats")
      elem = browser.find_element_by_tag_name("body")

      no_of_pagedowns = 3
      while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        no_of_pagedowns-=1


      ti=browser.find_elements_by_class_name("ChartHalfdonut__wrapper")
      for texts in ti:
        temp=texts.text.split("\n")
        worksheet_stats.write(row_stat, column_stat, temp[0])
        column_stat+=1
        worksheet_stats.write(row_stat, column_stat, temp[1])
        column_stat+=1
        worksheet_stats.write(row_stat, column_stat, temp[2])
        column_stat=0
        row_stat+=1
      workbook_stats.close()

      files.download("Stats"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("Scorer"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("CommentaireMatch"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("important"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("avant_match"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("phrase_but"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("resume"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")

    except:
      print(idx_match)    
    

In [6]:
'''def create_dataset_Commentaire_Sans_Resume(url,num_match,year):

  ### WEBDRIVER OPTIONS ###
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--ignore-certificate-errors')
  options.add_argument("--test-type")
  options.add_argument('--disable-browser-side-navigation')
  browser = webdriver.Chrome('chromedriver',options=options)
  

  for i, idx_match in zip(url, range(len(url))) :

      row_important = 0
      row = 0
      col = 0
      commentaire_avant_match = False
      row_commentaire_avant_match = 0
      index_match_equipe_saison = '-'.join(i.split('/')[6:])

      ### EXCEL FILES CREATION ###
      workbook = xlsxwriter.Workbook("CommentaireMatch"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_stats = xlsxwriter.Workbook("Stats"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_scorer = xlsxwriter.Workbook("Scorer"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_important = xlsxwriter.Workbook("important"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_avant_match = xlsxwriter.Workbook("avant_match"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_resume = xlsxwriter.Workbook("resume"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      workbook_phrase_but = xlsxwriter.Workbook("phrase_but"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")

      print("Stats"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")

      worksheet = workbook.add_worksheet()
      worksheet_stats = workbook_stats.add_worksheet()
      worksheet_scorer = workbook_scorer.add_worksheet()
      worksheet_important = workbook_important.add_worksheet()
      worksheet_avant_match = workbook_avant_match.add_worksheet()
      worksheet_resume = workbook_resume.add_worksheet()
      worksheet_phrase_but = workbook_phrase_but.add_worksheet()

      browser.get(i)
      time.sleep(1)
      compteur = 1

      ### REGULAR EXPRESSIONS ###
      a = 0
      apo='\''
      empty=''
      plus = "+"
      ex = "*"
      note=r'[0-9]+\/.[0-9]+'
      twitter=r'.*(PM).*(Twitter)?.*'
      compt = 0
      regex_time=r'[0-9]+\'\s?\+?[0-9]*'
      regex_if=r'\w'
      data={}
      data["moments"]=[]
      reg=r'.*But+.*'
      mid=r'45+?.*'
      regex_score_equipe1=r'([A-Z]+.*[0-9]’)'
      regex_score_equipe2=r'([0-9]+’.*[A-Z]+.*)'
      ### WEB PAGE NAVIGATION ###
      elem = browser.find_element_by_tag_name("body")
      no_of_pagedowns = 35
      while no_of_pagedowns:
          elem.send_keys(Keys.PAGE_DOWN)
          time.sleep(0.3)
          no_of_pagedowns-=1

      ### DATA LIST CREATION ###
      info = []
      info_stat = []
      info_score = []

      

      telem = browser.find_elements_by_class_name("TeamScore__identity")
      #equipe_dom= ""
      #equipe_ext= ""
      if len(telem)>0:
        equipe_dom = telem[0].text
        equipe_ext = telem[1].text
        if equipe_dom == "PSG":
          equipe_dom= "Paris"
        if equipe_ext == "PSG":
          equipe_ext= "Paris"
      else :
        equipe_dom, equipe_ext = ren_equipe(i)
      #SCORERS
      row_score1 = 0
      row_score2 = 0
      test= browser.find_elements_by_class_name("Scoreboard__goals")
      for texts in test:
        if(re.match(regex_score_equipe1,texts.text)):
          worksheet_scorer.write(row_score1, 0, texts.text)
          spli = texts.text.split(" ")
          if len(spli)==3:
            sentence = spli[0] + " " + spli[1] + " a marqué un but pour " + equipe_dom + " à la " + spli[2] + "."
          if len(spli)==2:
            sentence = spli[0] +  " a marqué un but pour " + equipe_dom + " à la " + spli[1] + "."
          worksheet_phrase_but.write(row_score1, 0, sentence)
          row_score1+=1
        if(re.match(regex_score_equipe2,texts.text)):
          worksheet_scorer.write(row_score2, 1, texts.text)
          spli = texts.text.split(" ")
          if len(spli)==3:
            sentence = spli[1] + " " + spli[2] + " a marqué un but pour " + equipe_ext + " à la " + spli[0] + "."
          if len(spli)==2:
            sentence = spli[1] +  " a marqué un but pour " + equipe_ext + " à la " + spli[0] + "."
          worksheet_phrase_but.write(row_score2, 1, sentence)
          row_score2+=1
      workbook_scorer.close()
      workbook_phrase_but.close()

      #GAME SCORE
      post_elems = browser.find_elements_by_class_name("TeamScore__team--winner") # TeamScore__score # TeamScore__team--looser # TeamScore__team--winner
      for texts in post_elems:
        info.append(texts.text)
      post_elems = browser.find_elements_by_class_name("TeamScore__team--looser") # TeamScore__score # TeamScore__team--looser # TeamScore__team--winner
      for texts in post_elems:
        info.append(texts.text) 
      if info != []:
        if info[0].isdigit():
          phrase = info[1] +" gagne le match contre " + info[2] + " sur un score de " + info[0]+"-"+info[3] + "."
        else :
          phrase = info[0] +" gagne le match contre " + info[3] + " sur un score de " + info[1]+"-"+info[2] + "."
      else :
        post_elems = browser.find_elements_by_class_name("TeamScore__name") 
        for texts in post_elems:
          info.append(texts.text)
        post_elems = browser.find_elements_by_class_name("TeamScore__score--ended") 
        for texts in post_elems:
          info.append(texts.text)
        phrase = info[0] +" fait match nul contre " + info[1] + " sur un score de " + info[2] + "."
      worksheet.write(row, col, phrase)
      row +=1

      #SENTENCES
      post_elems = browser.find_elements_by_class_name("grid__item")
      for texts in post_elems:
        if compteur >=0:
          if texts.text == "" or re.search(r'.*Notez.*',texts.text) or re.search(r'Top du match.*',texts.text) or re.search(r'Flop du match.*',texts.text) or re.search(r'.*note.*',texts.text) or re.search(r'.*[?].*',texts.text) or re.search(r'.*Twitter.*',texts.text): 
            a = a +1
          el= texts.find_elements_by_tag_name("img") 
          for em in el:
            if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/but.svg"  or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/carton_rouge.svg" or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/but_contre_son_camp.svg":
              worksheet_important.write(row_important, col, texts.text)
              #print("IMPORTANT: ",texts.text)
              row_important +=1
            if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/sifflet_fin_2mt.svg":
              worksheet_resume.write(0, 0, texts.text.split("\n")[2])
            if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/sifflet_debut_1mt.svg":
              #print("DEBUT: ",texts.text)
              commentaire_avant_match = True
          if commentaire_avant_match == True :
            worksheet_avant_match.write(row_commentaire_avant_match, col, texts.text)
            #print(texts.text)
            row_commentaire_avant_match += 1
          else :
            worksheet.write(row, col, texts.text)
            #print(texts.text)
            row +=1
        compteur +=1
                             
      workbook.close()
      workbook_important.close()
      workbook_resume.close()
      workbook_avant_match.close()
      #STATISTICS
      row_stat = 0
      column_stat = 0

      browser.get(i+"#stats")
      elem = browser.find_element_by_tag_name("body")

      no_of_pagedowns = 3
      while no_of_pagedowns:
          elem.send_keys(Keys.PAGE_DOWN)
          time.sleep(0.3)
          no_of_pagedowns-=1


      ti=browser.find_elements_by_class_name("ChartHalfdonut__wrapper")
      for texts in ti:
        temp=texts.text.split("\n")
        worksheet_stats.write(row_stat, column_stat, temp[0])
        column_stat+=1
        worksheet_stats.write(row_stat, column_stat, temp[1])
        column_stat+=1
        worksheet_stats.write(row_stat, column_stat, temp[2])
        column_stat=0
        row_stat+=1
      workbook_stats.close()

      files.download("Stats"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("Scorer"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("CommentaireMatch"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("important"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("avant_match"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("phrase_but"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
      files.download("resume"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")

    '''

'def create_dataset_Commentaire_Sans_Resume(url,num_match,year):\n\n  ### WEBDRIVER OPTIONS ###\n  options = webdriver.ChromeOptions()\n  options.add_argument(\'--headless\')\n  options.add_argument(\'--no-sandbox\')\n  options.add_argument(\'--disable-dev-shm-usage\')\n  options.add_argument(\'--ignore-certificate-errors\')\n  options.add_argument("--test-type")\n  options.add_argument(\'--disable-browser-side-navigation\')\n  browser = webdriver.Chrome(\'chromedriver\',options=options)\n  \n\n  for i, idx_match in zip(url, range(len(url))) :\n\n      row_important = 0\n      row = 0\n      col = 0\n      commentaire_avant_match = False\n      row_commentaire_avant_match = 0\n      index_match_equipe_saison = \'-\'.join(i.split(\'/\')[6:])\n\n      ### EXCEL FILES CREATION ###\n      workbook = xlsxwriter.Workbook("CommentaireMatch"+\'-\'+str(idx_match)+\'-\'+str(index_match_equipe_saison)+".xlsx")\n      workbook_stats = xlsxwriter.Workbook("Stats"+\'-\'+str(idx_match)+\'-\'+str(inde

In [7]:
!pip install nested-lookup
from nested_lookup import nested_lookup

  Created wheel for nested-lookup: filename=nested_lookup-0.2.21-cp36-none-any.whl size=7577 sha256=a393932364cc618fc1330a5ad0d996a50bca1f32ee8c818df0eb0bb7d6ea28df
  Stored in directory: /root/.cache/pip/wheels/cd/b4/f1/764f97bfa4c8a26e44eb2dc988e0ac8e918e54d9858ae6a349
Successfully built nested-lookup


In [0]:
import string
import json

In [0]:
def Get_Day(url):
  match_url = []
  resp = requests.get(url)
  soup = BeautifulSoup(resp.text, 'lxml')
  start = r'value="'  #a data-v-74b9a49b href="
  end = r'">\n'
  #i=0
  for link in soup.find('select', {'class': 'SelectNav__select'}):
    #print(link)#data-v-74b9a49b
    #i+=1
    #print(i)
    try :
      result = re.sub(r'.+value="','',str(link))
      result2 = re.sub(r'">(.*)\n*(.*)\n*(.*)','',str(result))
      #print(result2)
      match_url.append(result2)
    except:
      pass
  return match_url

In [0]:
def Get_Match(list_day):
  match_url = []
  for link in list_day:  
    #print(link)
    resp = requests.get(link)
    soup = BeautifulSoup(resp.text, 'lxml')
    #print(soup)
    res=re.sub(r'<html><body><p>','',str(soup))
    res2= re.sub(r'</p></body></html>','',res)
    jsonfile=json.loads(res2)
    try :
      b=nested_lookup("lien_web", jsonfile)
      for lien in b:
        a=lien.replace("webview","www")
      #print(result2)
        match_url.append(a)
    except:
      pass
  return match_url

In [0]:
def Get_all_txt(days_url,year):
  compt=0
  try:
    create_dataset_Commentaire_Sans_Resume(Get_Match(days_url), compt,year)     
  except:
    create_dataset_Commentaire_Sans_Resume(Get_Match(days_url), compt,year) 


In [0]:
def Get_all_txt(days_url,year):
  compt=0
  create_dataset_Commentaire_Sans_Resume(Get_Match(days_url), compt,year)     

In [13]:
Get_all_txt(Get_Day("https://www.lequipe.fr/Football/2018-2019/ligue-1-resultats.html#1ere-journee"),2019)

  0%|          | 1/380 [00:15<1:39:29, 15.75s/it]

0


  1%|          | 2/380 [00:28<1:33:39, 14.87s/it]

1


  1%|          | 3/380 [00:40<1:28:16, 14.05s/it]

...

131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210


 82%|████████▏ | 310/380 [12:27<03:07,  2.68s/it]

211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333


 97%|█████████▋| 369/380 [12:27<00:20,  1.87s/it]

334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379


In [0]:
Get_Match(Get_Day("https://www.lequipe.fr/Football/2018-2019/ligue-1-resultats.html#1ere-journee"))[21]

'https://www.lequipe.fr/Football/match-direct/ligue-1/2018-2019/angers-psg-live/428494'

In [0]:
wd = webdriver.Chrome('chromedriver',options=options)
wd.get("https://www.lequipe.fr/Football/match-direct/ligue-1/2018-2019/reims-psg-live/428521")
#print(wd.page_source)

elem = wd.find_element_by_tag_name("body")
no_of_pagedowns = 35
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.3)
    no_of_pagedowns-=1




telem = wd.find_elements_by_class_name("TeamScore__identity")
equipe_dom = telem[0].text
equipe_ext = telem[1].text

print("fight",equipe_dom, equipe_ext)

post_elems = wd.find_elements_by_class_name("grid__item")
for texts in post_elems:
  el= texts.find_elements_by_tag_name("img")
  
  for em in el:
    if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/but.svg" or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/sifflet_fin_2mt.svg" or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/carton_rouge.svg" or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/but_contre_son_camp.svg":
      print(texts.text)


fight Reims PSG
90'+6
Coup de sifflet final
Pour sa dernière à domicile, Reims s'offre le PSG et arrache la 8e place, son meilleur classement final dans l'élite depuis la saison 1975/76. Baba, Cafaro et Chavarria ont été décisifs dans ce match de gala. Le PSG concède un 5e revers en L1. Mbappé termine meilleur buteur du championnat (33 buts).
90'+4
But de Chavarria !
Reims crucifie le PSG ! Parti de sa moitié de terrain, Chavarria récupère le ballon côté gauche sur un long dégagement de Baba. L'attaquant rémois s'en va déborder Buffon par la droite et conclure du pied droit à ras de terre dans le but vide.
P. Chavarria
attaquant
59'
But de Mbappé !
Le PSG réduit l'écart ! A la réception d'un centre de Di Maria, Cavani dépose une belle tête pour l'extérieur pied droit de Mbappé dans les 6 mètres. Mendy est impuissant face à l'attaquant parisien.
K. Mbappé
attaquant
56'
But de Cafaro !
Reims double la mise ! Sollicité dans le dos de Kurzawa à droite dans la surface, Cafaro croise une mag

In [0]:
!ls

In [0]:
def Get_all_txt_Plus(days_url,year):
  compt = 0
  for i in Get_Match(days_url) :
    if compt > 308 :
      try:
        create_dataset(i, compt,year)     
      except:
        create_dataset(i, compt,year) 
    compt = compt + 1  

In [0]:
day_url2019 = []
day_url2019.append("https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/1re-journee")
for i in range(2,13):
  day_url2019.append("https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/"+str(i)+"e-journee")

In [0]:
day_url2018 = []
day_url2018.append("https://www.lequipe.fr/Football/2018-2019/ligue-1-resultats.html#1ere-journee")
for i in range(2,39):
  day_url2018.append("https://www.lequipe.fr/Football/2018-2019/ligue-1-resultats.html#"+str(i)+"e-journee")

In [0]:
day_url2017 = []
day_url2017.append("https://www.lequipe.fr/Football/2017-2018/ligue-1-resultats.html#1ere-journee")
for i in range(2,39):
  day_url2017.append("https://www.lequipe.fr/Football/2017-2018/ligue-1-resultats.html#"+str(i)+"e-journee")

In [0]:
day_url2016 = []
day_url2016.append("https://www.lequipe.fr/Football/2016-2017/ligue-1-resultats.html#1ere-journee")
for i in range(2,39):
  day_url2016.append("https://www.lequipe.fr/Football/2016-2017/ligue-1-resultats.html#"+str(i)+"e-journee")

In [0]:
day_url2015 = []
day_url2015.append("https://www.lequipe.fr/Football/2015-2016/ligue-1-resultats.html#1ere-journee")
for i in range(2,39):
  day_url2015.append("https://www.lequipe.fr/Football/2015-2016/ligue-1-resultats.html#"+str(i)+"e-journee")

In [0]:
day_url2014 = []
day_url2014.append("https://www.lequipe.fr/Football/2014-2015/ligue-1-resultats.html#1ere-journee")
for i in range(2,39):
  day_url2014.append("https://www.lequipe.fr/Football/2014-2015/ligue-1-resultats.html#"+str(i)+"e-journee")

In [0]:
day_url2013 = []
day_url2013.append("https://www.lequipe.fr/Football/2013-2014/ligue-1-resultats.html#1ere-journee")
for i in range(2,39):
  day_url2013.append("https://www.lequipe.fr/Football/2013-2014/ligue-1-resultats.html#"+str(i)+"e-journee")

In [0]:
day_url2019

['https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/1re-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/2e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/3e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/4e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/5e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/6e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/7e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/8e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/9e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/10e-journee',
 'https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/11e-journee']

In [0]:
def create_dataset(url,num_match,year):
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  # open it, go to a website, and get results
  options.add_argument('--ignore-certificate-errors')
  options.add_argument("--test-type")
  options.add_argument('--disable-browser-side-navigation')
  browser = webdriver.Chrome('chromedriver',options=options)
  workbook = xlsxwriter.Workbook("CommentaireMatch.xlsx")
  worksheet = workbook.add_worksheet()
  row = 0
  col = 0
  for i in url :
    print(i)
    browser.get(i)
    #WebDriverWait(browser, 5,2)
    time.sleep(1)
    #for post in post_elems:
     #   print(post.text)

    apo='\''
    empty=''
    plus = "+"
    ex = "*"
    note=r'[0-9]+\/.[0-9]+'
    twitter=r'.*(PM).*(Twitter)?.*'
    compt = 0
    regex_time=r'[0-9]+\'\s?\+?[0-9]*'
    regex_if=r'\w'
    data={}
    data["moments"]=[]
    reg=r'.*But+.*'
    mid=r'45+?.*'
    elem = browser.find_element_by_tag_name("body")

    no_of_pagedowns = 25
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)
        no_of_pagedowns-=1
    try:
      post_elems = browser.find_elements_by_class_name("grid__summary")
      for texts in post_elems:
        #print(texts.text)
        worksheet.write(row, col, texts.text)
        row +=1
        #file.write(texts.text)

      #workbook.close()
    except:
      create_dataset(url, num_match)
  workbook.close()
  files.download("CommentaireMatch.xlsx")
  #print(str(num_match)+" done")
  

In [0]:
def create_dataset_Commentaire_Sans_Resume(url,num_match,year):

  ### WEBDRIVER OPTIONS ###
  options = webdriver.ChromeOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--ignore-certificate-errors')
  options.add_argument("--test-type")
  options.add_argument('--disable-browser-side-navigation')
  browser = webdriver.Chrome('chromedriver',options=options)
  
  
  row = 0
  col = 0
  for i in url :

    ### EXCEL FILES CREATION ###
    workbook = xlsxwriter.Workbook("CommentaireMatch.xlsx")
    workbook_stats = xlsxwriter.Workbook("Stats.xlsx")
    workbook_scorer = xlsxwriter.Workbook("Scorer.xlsx")
    worksheet = workbook.add_worksheet()
    worksheet_stats = workbook_stats.add_worksheet()
    worksheet_scorer = workbook_scorer.add_worksheet()


    browser.get(i)
    time.sleep(1)
    compteur = 1

    ### REGULAR EXPRESSIONS ###
    a = 0
    apo='\''
    empty=''
    plus = "+"
    ex = "*"
    note=r'[0-9]+\/.[0-9]+'
    twitter=r'.*(PM).*(Twitter)?.*'
    compt = 0
    regex_time=r'[0-9]+\'\s?\+?[0-9]*'
    regex_if=r'\w'
    data={}
    data["moments"]=[]
    reg=r'.*But+.*'
    mid=r'45+?.*'
    regex_score_equipe1=r'([A-Z]+.*[0-9]’)'
    regex_score_equipe2=r'([0-9]+’.*[A-Z]+.*)'
    ### WEB PAGE NAVIGATION ###
    elem = browser.find_element_by_tag_name("body")
    no_of_pagedowns = 35
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.3)
        no_of_pagedowns-=1

    ### DATA LIST CREATION ###
    info = []
    info_stat = []
    info_score = []

    try:

      #SCORERS
      row_score1 = 0
      row_score2 = 0
      test= browser.find_elements_by_class_name("Scoreboard__goals")
      for texts in test:
        if(re.match(regex_score_equipe1,texts.text)):
          worksheet_scorer.write(row_score1, 0, texts.text)
          row_score1+=1
        if(re.match(regex_score_equipe2,texts.text)):
          worksheet_scorer.write(row_score2, 1, texts.text)
          row_score2+=1
      workbook_scorer.close()
      

      #GAME SCORE
      post_elems = browser.find_elements_by_class_name("TeamScore__team--winner") # TeamScore__score # TeamScore__team--looser # TeamScore__team--winner
      for texts in post_elems:
        info.append(texts.text)
      post_elems = browser.find_elements_by_class_name("TeamScore__team--looser") # TeamScore__score # TeamScore__team--looser # TeamScore__team--winner
      for texts in post_elems:
        info.append(texts.text) 
      if info != []:
        if info[0].isdigit():
          phrase = info[1] +" gagne le match contre " + info[2] + " avec un score " + info[0]+"-"+info[3]
        else :
          phrase = info[0] +" gagne le match contre " + info[3] + " avec un score " + info[1]+"-"+info[2]
      else :
        post_elems = browser.find_elements_by_class_name("TeamScore__name") 
        for texts in post_elems:
          info.append(texts.text)
        post_elems = browser.find_elements_by_class_name("TeamScore__score--ended") 
        for texts in post_elems:
          info.append(texts.text)
        phrase = info[0] +" fait match nul contre " + info[1] + " avec un score " + info[2]
      worksheet.write(row, col, phrase)
      row +=1

      #SENTENCES
      post_elems = browser.find_elements_by_class_name("grid__item")
      for texts in post_elems:
        if compteur >=0:
          if texts.text == "" or re.search(r'.*Notez.*',texts.text) or re.search(r'Top du match.*',texts.text) or re.search(r'Flop du match.*',texts.text) or re.search(r'.*note.*',texts.text) or re.search(r'.*[?].*',texts.text) or re.search(r'.*Twitter.*',texts.text): 
            a = a +1
          el= texts.find_elements_by_tag_name("img") 
          for em in el:
            if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/but.svg" or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/sifflet_fin_2mt.svg" or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/carton_rouge.svg" or em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/but_contre_son_camp.svg":
              worksheet.write(row, col, texts.text)
              print("IMPORTANT: ",texts.text)
              worksheet.write(row, col+1, "IMPORTANT")
              row +=1
            if em.get_attribute('src')=="https://www.lequipe.fr/img/icons/live/sifflet_debut_1mt.svg":
              worksheet.write(row, col, texts.text)
              print("DEBUT: ",texts.text)
              worksheet.write(row, col+1, "DEBUT")
              row +=1
          else :
            worksheet.write(row, col, texts.text)
            print(texts.text)
            row +=1
        compteur +=1
                             
      workbook.close()

      #STATISTICS
      row_stat = 0
      column_stat = 0

      browser.get(i+"#stats")
      elem = browser.find_element_by_tag_name("body")

      no_of_pagedowns = 3
      while no_of_pagedowns:
          elem.send_keys(Keys.PAGE_DOWN)
          time.sleep(0.3)
          no_of_pagedowns-=1


      ti=browser.find_elements_by_class_name("ChartHalfdonut__wrapper")
      for texts in ti:
        temp=texts.text.split("\n")
        worksheet_stats.write(row_stat, column_stat, temp[0])
        column_stat+=1
        worksheet_stats.write(row_stat, column_stat, temp[1])
        column_stat+=1
        worksheet_stats.write(row_stat, column_stat, temp[2])
        column_stat=0
        row_stat+=1
      workbook_stats.close()

    except:
      create_dataset_Commentaire_Sans_Resume(url, num_match,year)
    #files.download("Stats.xlsx")
    #files.download("Scorer.xlsx")
    #files.download("CommentaireMatch.xlsx")
    

In [0]:
saison = ["https://www.lequipe.fr/Football/ligue-1/page-calendrier-resultats/1re-journee","https://www.lequipe.fr/Football/2018-2019/ligue-1-resultats.html#1ere-journee","https://www.lequipe.fr/Football/2017-2018/ligue-1-resultats.html#1ere-journee","https://www.lequipe.fr/Football/2016-2017/ligue-1-resultats.html#1ere-journee","https://www.lequipe.fr/Football/2015-2016/ligue-1-resultats.html#1ere-journee","https://www.lequipe.fr/Football/2014-2015/ligue-1-resultats.html#1ere-journee","https://www.lequipe.fr/Football/2013-2014/ligue-1-resultats.html#1ere-journee"]

In [0]:
days_url = []
for i in saison:
  for j in Get_Day(i):
    days_url.append(j)

In [0]:
equipe = []
for i in Get_Match(days_url):
  i_split = i.split('/')
  i_equipe = i_split[7].split('-')
  if len(i_equipe) > 3:
    if i_equipe[0] == 'paris' or i_equipe[0] == 'saint' or i_equipe[0] == 'ac':
      del i_equipe[1]
    if i_equipe[1] == 'paris' or i_equipe[1] == 'saint' or i_equipe[1] == 'ac':
      del i_equipe[2]
  for j in range(2):
    if i_equipe[j] == 'ol': i_equipe[j] = 'lyon'
    elif i_equipe[j] == 'psg': i_equipe[j] = 'paris'
    elif i_equipe[j] == 'saint'or i_equipe[j] == 'asse': i_equipe[j] = 'saint etienne'
    elif i_equipe[j] == 'ac'or i_equipe[j] == 'gfc': i_equipe[j] = 'ajaccio'
    elif i_equipe[j] == 'om': i_equipe[j] = 'marseille'
  equipe.append(i_equipe[0])
  equipe.append(i_equipe[1])


In [0]:
unique_equipe = set(equipe)
unique_equipe_list = list(unique_equipe)

In [0]:
unique_equipe_list

['lorient',
 'troyes',
 'toulouse',
 'lyon',
 'brest',
 'bordeaux',
 'evian',
 'metz',
 'montpellier',
 'saint',
 'nice',
 'marseille',
 'amiens',
 'nantes',
 'rennes',
 'nancy',
 'lens',
 'guingamp',
 'angers',
 'nimes',
 'sochaux',
 'dijon',
 'reims',
 'strasbourg',
 'valenciennes',
 'paris',
 'lille',
 'bastia',
 'caen',
 'monaco',
 'ajaccio',
 'thonon']

In [0]:
unique_equipe_list

['bordeaux',
 'strasbourg',
 'lyon',
 'monaco',
 'thonon',
 'valenciennes',
 'paris',
 'reims',
 'om',
 'dijon',
 'etienne',
 'nantes',
 'nice',
 'brest',
 'ajaccio',
 'sg',
 'psg',
 'toulouse',
 'ol',
 'metz',
 'guingamp',
 'saint',
 'lens',
 'angers',
 'asse',
 'evian',
 'montpellier',
 'amiens',
 'nancy',
 'gfc',
 'ac',
 'caen',
 'lille',
 'lorient',
 'marseille',
 'bastia',
 'nimes',
 'troyes',
 'rennes',
 'sochaux']

In [0]:
['bordeaux',
 'strasbourg',
 'lyon', 'ol', --- > lyon
 'monaco',
 'thonon',
 'valenciennes',
 'paris', 'sg', 'psg', ---> paris
 'reims',
 'om', ---> marseille
 'dijon',
 'etienne', 'saint', 'asse', --> saint-etienne
 'nantes',
 'nice',
 'brest',
 'ajaccio', 'ac', 'gfc', ---> ajaccio
 'toulouse',
 'metz',
 'guingamp',
 'lens',
 'angers',
 'evian',
 'montpellier',
 'amiens',
 'nancy',
 'caen',
 'lille',
 'lorient',
 'marseille',
 'bastia',
 'nimes',
 'troyes',
 'rennes',
 'sochaux'

In [0]:
def ren_equipe(url):
  equipe = []
  i_split = url.split('/')
  i_equipe = i_split[7].split('-')
  if len(i_equipe) > 3:
    if i_equipe[0] == 'paris' or i_equipe[0] == 'saint' or i_equipe[0] == 'ac':
      del i_equipe[1]
    if i_equipe[1] == 'paris' or i_equipe[1] == 'saint' or i_equipe[1] == 'ac':
      del i_equipe[2]
  for j in range(2):
    if i_equipe[j] == 'ol': i_equipe[j] = 'lyon'
    elif i_equipe[j] == 'psg': i_equipe[j] = 'paris'
    elif i_equipe[j] == 'saint'or i_equipe[j] == 'asse': i_equipe[j] = 'saint etienne'
    elif i_equipe[j] == 'ac'or i_equipe[j] == 'gfc': i_equipe[j] = 'ajaccio'
    elif i_equipe[j] == 'om': i_equipe[j] = 'marseille'
  equipe.append(i_equipe[0][0].upper()+i_equipe[0][1:])
  equipe.append(i_equipe[1][0].upper()+i_equipe[1][1:])
  return equipe

In [0]:
m,n=ren_equipe("https://www.lequipe.fr/Football/match-direct/ligue-1/2019-2020/dijon-asse-live/450972")

In [0]:

### WEBDRIVER OPTIONS ###
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")
options.add_argument('--disable-browser-side-navigation')
browser = webdriver.Chrome('chromedriver',options=options)


row = 0
col = 0
i = "https://www.lequipe.fr/Football/match-direct/ligue-1/2018-2019/angers-psg-live/428494"
row_important = 0
row = 0
col = 0
commentaire_avant_match = False
row_commentaire_avant_match = 0
index_match_equipe_saison = '-'.join(i.split('/')[6:])

### EXCEL FILES CREATION ###
workbook = xlsxwriter.Workbook("CommentaireMatch"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
workbook_stats = xlsxwriter.Workbook("Stats"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
workbook_scorer = xlsxwriter.Workbook("Scorer"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
workbook_important = xlsxwriter.Workbook("important"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
workbook_avant_match = xlsxwriter.Workbook("avant_match"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
workbook_resume = xlsxwriter.Workbook("resume"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")
workbook_phrase_but = xlsxwriter.Workbook("phrase_but"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")

print("Stats"+'-'+str(idx_match)+'-'+str(index_match_equipe_saison)+".xlsx")

worksheet = workbook.add_worksheet()
worksheet_stats = workbook_stats.add_worksheet()
worksheet_scorer = workbook_scorer.add_worksheet()
worksheet_important = workbook_important.add_worksheet()
worksheet_avant_match = workbook_avant_match.add_worksheet()
worksheet_resume = workbook_resume.add_worksheet()
worksheet_phrase_but = workbook_phrase_but.add_worksheet()

browser.get(i)
time.sleep(1)
compteur = 1

### REGULAR EXPRESSIONS ###
a = 0
apo='\''
empty=''
plus = "+"
ex = "*"
note=r'[0-9]+\/.[0-9]+'
twitter=r'.*(PM).*(Twitter)?.*'
compt = 0
regex_time=r'[0-9]+\'\s?\+?[0-9]*'
regex_if=r'\w'
data={}
data["moments"]=[]
reg=r'.*But+.*'
mid=r'45+?.*'
regex_score_equipe1=r'([A-Z]+.*[0-9]’)'
regex_score_equipe2=r'([0-9]+’.*[A-Z]+.*)'
### WEB PAGE NAVIGATION ###
elem = browser.find_element_by_tag_name("body")
no_of_pagedowns = 35
while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.3)
    no_of_pagedowns-=1

### DATA LIST CREATION ###
info = []
info_stat = []
info_score = []



telem = browser.find_elements_by_class_name("TeamScore__identity")
#equipe_dom= ""
#equipe_ext= ""
if len(telem)>0:
  equipe_dom = telem[0].text
  equipe_ext = telem[1].text
  if equipe_dom == "PSG":
    equipe_dom= "Paris"
  if equipe_ext == "PSG":
    equipe_ext= "Paris"
else :
  equipe_dom, equipe_ext = ren_equipe(i)
#SCORERS
row_score1 = 0
row_score2 = 0
test= browser.find_elements_by_class_name("Scoreboard__goals")
for texts in test:
  if(re.match(regex_score_equipe1,texts.text)):
    worksheet_scorer.write(row_score1, 0, texts.text)
    spli = texts.text.split(" ")
    sentence = spli[0] + " " + spli[1] + " a marqué un but pour " + equipe_dom + " à la " + spli[2] + "."
    worksheet_phrase_but.write(row_score1, 0, sentence)
    row_score1+=1
  if(re.match(regex_score_equipe2,texts.text)):
    worksheet_scorer.write(row_score2, 1, texts.text)
    spli = texts.text.split(" ")
    sentence = spli[1] + " " + spli[2] + " a marqué un but pour " + equipe_ext + " à la " + spli[0] + "."
    worksheet_phrase_but.write(row_score2, 1, sentence)
    row_score2+=1
workbook_scorer.close()
workbook_phrase_but.close()


Stats-10-2018-2019-angers-psg-live-428494.xlsx


IndexError: ignored